In [2]:
import cv2
from keras.models import load_model
import numpy as np

#recognizer = cv2.face.LBPHFaceRecognizer_create()         # 啟用訓練人臉模型方法
#recognizer.read('face.yml')                               # 讀取人臉模型檔
cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'  # 載入人臉追蹤模型
face_cascade = cv2.CascadeClassifier(cascade_path)    

# 打开摄像头
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("无法打开摄像头")
    exit()

# 加载模型
model = load_model('new_model.h5')

# 定义类别标签
# 假设你的模型是一个10类分类器，这里给出示例标签

list = ['Bulbasaur','Charmander','Squirtle','Chikorita','Cyndaquil','Totodile','Treecko','Torchic','Mudkip','Turtwig','Chimchar','Piplup','Snivy','Tepig','Oshawott','Chespin','Fennekin','Froakie','Rowlet','Litten','Popplio','Grookey','Scorbunny','Sobble','Sprigatito','Fuecoco','Quaxly']
class_labels = [f'{list[i]}({i})'for i in range(27)]   #標籤

while True:
    # 捕获帧
    ret, frame = cap.read()
    if not ret:
        print("无法接收帧（流结束？）。退出...")
        break
        
 # 依序判斷每張臉屬於哪個 id
    frame = cv2.resize(frame,(500,500))            # 縮小尺寸，加快辨識效率
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  # 轉換成黑白
    faces = face_cascade.detectMultiScale(gray)  # 追蹤人臉 ( 目的在於標記出外框 )
    for(x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            # 標記人臉外框
    #裁剪並預處理人臉
    face = frame[y:y+h, x:x+w]
    face = cv2.resize(face, (256, 256))  # 根據你的模型輸入大小調整
    face = face.astype('float32') / 255.0
    face = np.expand_dims(face, axis=0)  # 增加批次維度
    
    # 将帧调整为模型所需的格式
    # resized_frame = cv2.resize(frame, (256, 256))  # 调整为所需大小 256x256
    # imgs = np.array([resized_frame]).astype('float32') / 255  # 归一化到 [0, 1] 区间
    
    # 进行预测
    predictions = model.predict(face)
    
    #找出最大值及其对应的类别
    for image_index in range(len(predictions)):
        top_prediction = predictions[image_index]
        max_index = np.argmax(top_prediction)
        max_value = top_prediction[max_index]
        predicted_label = class_labels[max_index]
        
        # 打印预测结果
        print(f"Image #{image_index}")
        print(f" {max_index} - {predicted_label:12s} {max_value * 100:.2f}%")
        print()
        
        # 在图像上显示预测结果
        cv2.putText(frame, f"predict: {predicted_label} ({max_value * 100:.2f}%)", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # 显示包含预测结果的帧
    cv2.imshow('live', frame)
    
    # 按下 'q' 键退出循环
    if cv2.waitKey(1) == ord('q'):
        break

# 释放摄像头并关闭所有 OpenCV 窗口
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 212ms/step
Image #0
 10 - Chimchar(10) 100.00%

1/1 [==============================] - 0s 30ms/step
Image #0
 18 - Rowlet(18)   57.31%

1/1 [==============================] - 0s 30ms/step
Image #0
 10 - Chimchar(10) 100.00%

1/1 [==============================] - 0s 29ms/step
Image #0
 18 - Rowlet(18)   51.99%

1/1 [==============================] - 0s 35ms/step
Image #0
 10 - Chimchar(10) 100.00%

1/1 [==============================] - 0s 32ms/step
Image #0
 10 - Chimchar(10) 99.99%

1/1 [==============================] - 0s 31ms/step
Image #0
 10 - Chimchar(10) 99.99%

1/1 [==============================] - 0s 31ms/step
Image #0
 18 - Rowlet(18)   50.14%

1/1 [==============================] - 0s 32ms/step
Image #0
 18 - Rowlet(18)   52.45%

1/1 [==============================] - 0s 33ms/step
Image #0
 10 - Chimchar(10) 99.99%

1/1 [==============================] - 0s 26ms/step
Image #0
 18 - Rowlet(18)   61.83%

1/1 [=======================